In [ ]:
import numpy as np
import pandas as pd
from scapy.all import IP, rdpcap

In [ ]:
def resize(arr, size):
    if len(arr) >= size:
        return arr[0:size]
    return np.pad(arr,(0,size - len(arr)))

In [ ]:
data_frame = list()
local_ip = '192.168.1.111'
resize_len = 10000
input_path = 'E:/4tab-traffic/'
count_repeat = 10

# 读取csv文件中的组合
combinations = pd.read_csv('E:/4taburl-label.csv')

for index, row in combinations.iterrows():
    A, B, C, D = row['A'], row['B'], row['C'], row['D']
    
    for F in range(count_repeat):
        file_name = f'{input_path}{int(A)}-{int(B)}-{int(C)}-{int(D)}-{F}.pcap'
        
        try:
            packets = rdpcap(file_name)
            data = [A, B, C, D, F]
            
            for packet in packets:
                if IP in packet:
                    ip_src = packet[IP].src
                    ip_dst = packet[IP].dst

                    if ip_src == local_ip:
                        data.append(len(packet))
                    elif ip_dst == local_ip:
                        data.append(-len(packet))

            data_list = np.array(data)
            data_list = resize(data_list, resize_len + 5)
            data_list = np.array([data_list])

            if len(data_frame) == 0:
                data_frame = data_list
            else:
                data_frame = np.append(data_frame, data_list, axis=0)
        except Exception as e:
            print(f'\tError opening file {file_name}:', e)
            pass

    print(f'\rprocessing: {A}-{B}-{C}-{D}', end='')

print('\ndone!')

out_path = 'E:/msedge/'
df_data = pd.DataFrame(data_frame)
df_data.to_csv(out_path + 'msedge-4tab.csv', index=None, header=None)
